In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        pass
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!ls

In [ ]:
#导库
import os
import pandas as pd
import numpy as np
import cv2
from PIL import Image, ImageDraw

import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
import plotly 
print(plotly.__version__)
#需要注意一下plotly 这里可能有坑

In [ ]:
!pwd #记得分清楚 working跟input跟output路径的区别！！
!ls
!ls /kaggle/input/nfl-health-and-safety-helmet-assignment

In [ ]:
# /kaggle/input/nfl-health-and-safety-helmet-assignment
ENV_DIR = '/kaggle/input'
DATA_DIR = f'{ENV_DIR}/nfl-health-and-safety-helmet-assignment'
#查看一下路径在哪
os.listdir(DATA_DIR) 
#os.path.exists(DATA_DIR+"/train_labels.csv") ＃判断是否存在路径

# 导入数据集

In [ ]:
#查看csv数据
#视频数据的标签
train_df = pd.read_csv(f"{DATA_DIR}/train_labels.csv")
train_df.head(2) 
#可以看到有很多标注参数信息 

# - video_frame: 视频和帧的组合编号
# - gameKey: game编号
# - playID: play编号
# - view: 摄像机视角
# - video: 视频编号
# - frame: 帧编号
# - label: 球员编码
# - [left/width/top/height]: 头盔边框
# - impactType: 头盔撞击类型的描述:头盔、肩膀、身体、地面
# - isDefinitiveImpact ：是否撞击的标签
# - isSidelinePlayer ：是都位于变现上

In [ ]:
#图片数据的标签
image_df = pd.read_csv(f"{DATA_DIR}/image_labels.csv")
image_df.head(2) 

# - image: 图像文件名。
# - label: 标签类型。
# - [left/width/top/height]：标签边界

In [ ]:
# 辅助数据集训练结果 - Trained by images inside folder images/
train_predict_df = pd.read_csv(f'{DATA_DIR}/train_baseline_helmets.csv')
test_predict_df = pd.read_csv(f'{DATA_DIR}/test_baseline_helmets.csv')
train_predict_df.head(2)
# - video_frame：视频和帧的组合编号
# - [left/width/top/height]：预测边框
# - conf ：边框置信度

In [ ]:
# 球员传感器数据
train_tracking_df = pd.read_csv(f'{DATA_DIR}/train_player_tracking.csv')
test_tracking_df = pd.read_csv(f'{DATA_DIR}/test_player_tracking.csv')
train_tracking_df.head(2)
# - gameKey: game编号.
# - playID:  球员编号.
# - player: the player's ID code.
# - time: timestamp at 10 Hz.
# - x: player position along the long axis of the field. See figure below.
# - y: player position along the short axis of the field. See figure below.
# - s: 速度 in yards/second.
# - a: 加速度 in yards/second^2.
# - dis: distance traveled from prior time point, in yards.
# - o: 球员方向角度 (deg).
# - dir: angle of player motion (deg).
# - event: game events like a snap, whistle, etc

In [ ]:
#检查submission格式是否正确
def check_submission(sub):
    """
    Checks that the submission meets all the requirements.

    1. No more than 22 Boxes per frame.
    2. Only one label prediction per video/frame
    3. No duplicate boxes per frame.

    Returns:
        True -> Passed the tests
        False -> Failed the test
    """
    # Maximum of 22 boxes per frame.
    max_box_per_frame = sub.groupby(["video_frame"])["label"].count().max()
    if max_box_per_frame > 22:
        print("Has more than 22 boxes in a single frame")
        return False
    # Only one label allowed per frame.
    has_duplicate_labels = sub[["video_frame", "label"]].duplicated().any()
    if has_duplicate_labels:
        print("Has duplicate labels")
        return False
    # Check for unique boxes
    has_duplicate_boxes = (
        sub[["video_frame", "left", "width", "top", "height"]].duplicated().any()
    )
    if has_duplicate_boxes:
        print("Has duplicate boxes")
        return False
    return True

In [ ]:
ans_submission = pd.read_csv(f'{DATA_DIR}/sample_submission.csv')
check_submission(ans_submission)

# 以抽视频祯的方式来得到img

In [ ]:
# reference : https://www.kaggle.com/coldfir3/eda-helmet-keypoint-tracking-data-comparison

def get_frame_from_video(video_path, frame):
    """
    video_path:视频路径
    frame: 哪一秒图片（以一秒为起点）
    """
    video_path = f"{DATA_DIR}/train/{video_path}"
    frame = frame - 1 
    
    !ffmpeg \
        -hide_banner \
        -loglevel fatal \
        -nostats \
        -i $video_path -vf "select=eq(n\,$frame)" -vframes 1 frame.png
    
    img = Image.open('frame.png') #打开图片
    os.remove('frame.png') #把图片删去
    return img

In [ ]:
get_frame_from_video('57583_000082_Endzone.mp4', 4) #以57583_000082_Endzone.mp4的四秒为例

# 画出图片上的box

In [ ]:
def draw_rect(image, bbox_df):
    """
    image：原始图片
    bbox_df:box的位置（由标注文件给出）
    """
    new_image = image.copy()
    draw = ImageDraw.Draw(new_image)
    for _, (left, width, top, height) in bbox_df[['left', 'width', 'top', 'height']].iterrows():
        # 画图由左上到右下，所以参数：(left, top), (left + width, top + height)
        draw.rectangle(((left, top), (left + width, top + height)), outline=(255, 0, 0), width=2)
    
    return new_image

In [ ]:
def frame_bbox(df, video_frame):
    """
    df:
    video_frame:输入参数 57583_000082_Endzone 加上 "_秒"
    """
    video_name = '_'.join(video_frame.split('_')[:3]) + '.mp4'
#     print(video_name)
    #string分割
    frame = int(video_frame.split('_')[-1])
#     print(frame)
    #调用上面的取祯方法
    image = get_frame_from_video(video_name, frame)
    #利用pandas的query查询（@可以引入环境变量）
    #https://blog.csdn.net/qq_45176548/article/details/112755795
    bbox_df = df.query('video_frame == @video_frame')
    #调用方法画出边框
    bbox_image = draw_rect(image, bbox_df)
    
    return bbox_image

In [ ]:
frame_bbox(train_df, '57583_000082_Endzone_4')

# 导入视频观看

In [ ]:
from IPython.display import Video, display
def video(video_path, ratio=0.5):
    """
    video_path:视频路径
    ratio:比例
    """
    nfl_video = Video(f"{DATA_DIR}/train/{video_path}",
                      embed=True,
                      height=int(720 * ratio),
                      width=int(1280 * ratio))
    return nfl_video
video('57583_000082_Endzone.mp4')

# 分析传感器资料，匹配上球员id与速度等信息，制作动画

In [ ]:
# 可参考：https://www.kaggle.com/robikscube/nfl-helmet-assignment-getting-started-guide
def add_track_features(tracks, fps=59.94, snap_frame=10):
    """
    Add column features helpful for syncing with video data.
    
    track:csv信息
    
    """
    tracks = tracks.copy()
    
    tracks["game_play"] = (
        tracks["gameKey"].astype("str")
        + "_"
        + tracks["playID"].astype("str").str.zfill(6)
    )
    #找到比赛时间
    tracks["time"] = pd.to_datetime(tracks["time"])
    # The time when snap happened
    # print(tracks.query('event == "ball_snap"').groupby("game_play")["time"].first())
    # 一个字典，由track（df）查询，然后转为dict
    snap_dict = (
        tracks.query('event == "ball_snap"')
        .groupby("game_play")["time"]
        .first()
        .to_dict()
    )
#     print(tracks["game_play"].map(snap_dict))
    # 目前还不太清楚snap是什么
    tracks["snap"] = tracks["game_play"].map(snap_dict)
    tracks["isSnap"] = tracks["snap"] == tracks["time"]
    #队员分成主队与客队
    tracks["team"] = tracks["player"].str[0].replace("H", "Home").replace("V", "Away")
    tracks["snap_offset"] = (tracks["time"] - tracks["snap"]).astype(
        "timedelta64[ms]"
    ) / 1_000
    # 视频祯
    tracks["est_frame"] = (
        ((tracks["snap_offset"] * fps) + snap_frame).round().astype("int")
    )
    # 看一眼数据
    print("game_play:",tracks["game_play"][0])
    print("snap:",tracks["snap"][0])
    print("team:",tracks["team"][0])
    print("isSnap:",tracks["isSnap"][0])
    print("snap_offset:",tracks["snap_offset"][0])
    
    return tracks

train_tracking_df = add_track_features(train_tracking_df)

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import plotly

#画出场地
def add_plotly_field(fig):
    # 参考 https://www.kaggle.com/ammarnassanalhajali/nfl-big-data-bowl-2021-animating-players
    fig.update_traces(marker_size=20)
    
    fig.update_layout(paper_bgcolor='#29a500', plot_bgcolor='#29a500', font_color='white',
        width = 800,
        height = 600,
        title = "",
        
        xaxis = dict(
        nticks = 10,
        title = "",
        visible=False
        ),
        
        yaxis = dict(
        scaleanchor = "x",
        title = "Temp",
        visible=False
        ),
        showlegend= True,
  
        annotations=[
       dict(
            x=-5,
            y=26.65,
            xref="x",
            yref="y",
            text="ENDZONE",
            font=dict(size=16,color="#e9ece7"),
            align='center',
            showarrow=False,
            yanchor='middle',
            textangle=-90
        ),
        dict(
            x=105,
            y=26.65,
            xref="x",
            yref="y",
            text="ENDZONE",
            font=dict(size=16,color="#e9ece7"),
            align='center',
            showarrow=False,
            yanchor='middle',
            textangle=90
        )]  
        ,
        legend=dict(
        traceorder="normal",
        font=dict(family="sans-serif",size=12),
        title = "",
        orientation="h",
        yanchor="bottom",
        y=1.00,
        xanchor="center",
        x=0.5
        ),
    )
    ####################################################
        
    fig.add_shape(type="rect", x0=-10, x1=0,  y0=0, y1=53.3,line=dict(color="#c8ddc0",width=3),fillcolor="#217b00" ,layer="below")
    fig.add_shape(type="rect", x0=100, x1=110, y0=0, y1=53.3,line=dict(color="#c8ddc0",width=3),fillcolor="#217b00" ,layer="below")
    for x in range(0, 100, 10):
        fig.add_shape(type="rect", x0=x,   x1=x+10, y0=0, y1=53.3,line=dict(color="#c8ddc0",width=3),fillcolor="#29a500" ,layer="below")
    for x in range(0, 100, 1):
        fig.add_shape(type="line",x0=x, y0=1, x1=x, y1=2,line=dict(color="#c8ddc0",width=2),layer="below")
    for x in range(0, 100, 1):
        fig.add_shape(type="line",x0=x, y0=51.3, x1=x, y1=52.3,line=dict(color="#c8ddc0",width=2),layer="below")
    
    for x in range(0, 100, 1):
        fig.add_shape(type="line",x0=x, y0=20.0, x1=x, y1=21,line=dict(color="#c8ddc0",width=2),layer="below")
    for x in range(0, 100, 1):
        fig.add_shape(type="line",x0=x, y0=32.3, x1=x, y1=33.3,line=dict(color="#c8ddc0",width=2),layer="below")
    
    
    fig.add_trace(go.Scatter(
    x=[2,10,20,30,40,50,60,70,80,90,98], y=[5,5,5,5,5,5,5,5,5,5,5],
    text=["G","1 0","2 0","3 0","4 0","5 0","4 0","3 0","2 0","1 0","G"],
    mode="text",
    textfont=dict(size=20,family="Arail"),
    showlegend=False,
    ))
    
    fig.add_trace(go.Scatter(
    x=[2,10,20,30,40,50,60,70,80,90,98], y=[48.3,48.3,48.3,48.3,48.3,48.3,48.3,48.3,48.3,48.3,48.3],
    text=["G","1 0","2 0","3 0","4 0","5 0","4 0","3 0","2 0","1 0","G"],
    mode="text",
    textfont=dict(size=20,family="Arail"),
    showlegend=False,
    ))
    
    return fig

In [ ]:
#经过add_track_features 修改的df
train_tracking_df
train_tracking_df.head(3)

In [ ]:
def football_animation(game_play):
    train_tracking_df["track_time_count"] = (
        train_tracking_df.sort_values("time")
        .groupby("game_play")["time"]
        .rank(method="dense")
        .astype("int")
    )

    fig = px.scatter(
        train_tracking_df.query("game_play == @game_play"),
        x="x",
        y="y",
        #调整大小
        range_x=[-10, 110],
        range_y=[-10, 53.3],
        hover_data=["player", "s", "a", "dir"],#参数
        color="team", #用主客队分类
        animation_frame="track_time_count",#动画的祯
        text="player",
        title=f"Animation of NGS data for game_play {game_play}",
    )

    fig.update_traces(textfont_size=10)
    fig = add_plotly_field(fig)
    fig.show()
football_animation('57583_000082')

In [ ]:
train_videos = os.listdir(f'{DATA_DIR}/train')
test_videos = os.listdir(f'{DATA_DIR}/test')
#训练视频与测试视频的数量
len(train_videos), len(test_videos)

In [ ]:
set(test_videos).issubset(set(train_videos))
#也就是test是train的子集，包含在里头
#有120个train_video与6个test_video

In [ ]:
test_df = train_df.query("video in @test_videos").reset_index().copy()
test_df

In [ ]:
#区分不同场地视角，endzone，sideline
end_count = 0
side_count = 0
endzone_list = []
sideline_list = []
for train_video in train_videos:
    name = train_video.split('.')[0]
    video_id, play_id, view = name.split('_')
    
    if view == "Endzone":
        endzone_list.append('_'.join([video_id, play_id]))
        end_count += 1
    else:
        sideline_list.append('_'.join([video_id, play_id]))
        side_count += 1

print(end_count, side_count)
#都各自有60个
print(set(train_videos) == set(train_df.video.unique()))
#代表train_videos 在train_labels.csv能找到
set(test_videos) == set(test_df.video.unique())

In [ ]:
#因为同时有两个不同场景的视角，需要分析不同视角是否有相同祯数
not_match_video = []

for play_id in train_df.playID.unique():
    end_frame_n = train_df.query('playID == @play_id and view == "Endzone"').frame.max()
    side_frame_n = train_df.query('playID == @play_id and view == "Sideline"').frame.max()
    
    if end_frame_n != side_frame_n:
        not_match_video.append(play_id)
        print(f'Not same at playID {play_id} endzone [{end_frame_n}] sideline [{side_frame_n}] difference [{abs(end_frame_n - side_frame_n)}]')
print("no_match_num:",len(not_match_video))
#有25个video不匹配祯数

In [ ]:
## 分析每个祯是否在csv都是能对的上的
def get_total_frame(video_path):
    """
    video_path:视频路径
    """
    cap = cv2.VideoCapture(f"{DATA_DIR}/train/{video_path}")
    property_id = int(cv2.CAP_PROP_FRAME_COUNT) 
    length = int(cv2.VideoCapture.get(cap, property_id))
    
    return length

play2frame = train_df.groupby('video').frame.max().to_dict()

for video_name, label_frame_n in play2frame.items():
    video_frame_n = get_total_frame(video_name)
    if video_frame_n != label_frame_n:
        print('Not Match!')

In [ ]:
#是否有frame为0的情况
train_df.query("frame == 0")
#只有一祯 frame标为0 （应该为1） =>直接舍弃即可
# 可以用这个看一下该视频的情况，发现只有一个为0所以直接舍弃
# frame_df = train_df.query('video == "57584_000336_Sideline.mp4"')
# frame_df
# frame_df.frame.max()
# get_total_frame("57584_000336_Sideline.mp4")

# 查看train_labels.csv！！

In [ ]:
train_df.head(2)

In [ ]:
print(train_df.gameKey.nunique())
print(train_df.playID.nunique())
print(train_df.groupby('gameKey')['playID'].unique())

In [ ]:
# 300 ~ 600 images per play
# 可以看一下'gameKey', 'playID', 'view'的区别
train_df.groupby(['gameKey', 'playID', 'view'])['frame'].nunique()

In [ ]:
#画图：祯数分布
sns.displot(train_df.groupby(['gameKey', 'playID', 'view'])['frame'].nunique().values)
print("total_frame:",train_df.groupby(['gameKey', 'playID', 'view'])['frame'].nunique().sum())

In [ ]:
play_per_game = train_df.groupby('gameKey')['playID'].nunique().reset_index().groupby('playID')['gameKey'].unique().to_dict()
play_per_game #game与playid的分类
# 一场game可能有几个playid 

In [ ]:
train_df.groupby(['gameKey', 'playID', 'view'])['label'].nunique().value_counts()
#每场比赛球场的人数

In [ ]:
#确认一下哪场比赛只有16人
train_df.groupby(['gameKey', 'playID', 'view'])['label'].nunique().reset_index().query('label == 16')

In [ ]:
video('57680_002206_Endzone.mp4')

In [ ]:
train_df.groupby(['gameKey', 'playID', 'view'])['label'].nunique().reset_index().query('label == 24')

In [ ]:
frame_bbox(train_df, '57997_003691_Sideline_4')
#标程24个框的其中一祯

In [ ]:
print("球员人数：",len(train_df.label.unique()))
train_df.label.unique()

In [ ]:
train_df['isHome'] = train_df.label.str[0]
train_df['num'] = train_df.label.str[1:].astype(int)
train_df[['isHome', 'num']]

In [ ]:
home_players = train_df.query('isHome == "H"').num.unique()
visitor_players = train_df.query('isHome == "V"').num.unique()

home_players.sort(), visitor_players.sort()
home_players, visitor_players

In [ ]:
print("主队人数：",len(home_players), "客队人数：",len(visitor_players))
print(set(home_players) - set(visitor_players)) #主场才有的背号
print(set(visitor_players) - set(home_players)) #客场才有的背号

In [ ]:
# 有边线球员被记录的？
sideline_play = train_df.query('isSidelinePlayer == True').playID.unique()
print("playid:",sideline_play,"数量：",len(sideline_play))

In [ ]:
# 包括边线球员的比赛中有多少球员？isSidelinePlayer == 2
sideline_df = train_df.groupby(['gameKey', 'playID', 'view'])['label', 'isSidelinePlayer'].nunique().query('isSidelinePlayer == 2')
sideline_df.head(2)
# sideline_df = train_df.groupby(['gameKey', 'playID', 'view'])['label', 'isSidelinePlayer'].nunique().query('isSidelinePlayer == 1')
# sideline_df.head(2)


In [ ]:
# 有25个Sideline看到边线球员，5个Endzone看到边线球员
sideline_df.reset_index().view.value_counts()

In [ ]:
sideline_df.label.value_counts()

# 分析头盔尺寸

In [ ]:
train_df['helmet_size'] = train_df.width * train_df.height
train_df.helmet_size.hist();

In [ ]:
# 头盔最多size的前10
train_df.helmet_size.value_counts()[:10]

In [ ]:
# 头盔最少size的前10个
train_df.helmet_size.value_counts()[-10:]

In [ ]:
# 最大尺寸的头盔，最小尺寸的头盔
train_df.helmet_size.max(), train_df.helmet_size.min()

In [ ]:
train_df.query('helmet_size == 5928')

In [ ]:
get_frame_from_video('57686_002546_Endzone.mp4', 429)
frame_bbox(train_df, '57686_002546_Endzone_429')

In [ ]:
train_df.query('helmet_size == 9')

In [ ]:
frame_bbox(train_df, '57680_002206_Sideline_149')

# 冲撞类型在数据中的影响

In [ ]:
# 看一下冲撞的类型
train_df.impactType.value_counts()

In [ ]:
# 冲撞
impact_index = train_df.query('impactType != "None"').index
impact_index

In [ ]:
# 决定性冲撞
definite_impact_index = train_df.query('isDefinitiveImpact == True').index
definite_impact_index

In [ ]:
# 冲撞不是决定冲撞的子集，但是决定冲撞是冲撞的子集
set(impact_index).issubset(set(definite_impact_index)), set(definite_impact_index).issubset(set(impact_index))

In [ ]:
# isDefinitiveImpact == True 的冲撞都算冲撞
train_df.query('isDefinitiveImpact == True').impactType.value_counts()

# 多久发生一次DefinitiveImpact

In [ ]:
train_df.isDefinitiveImpact = train_df.isDefinitiveImpact.astype(int)
# train_df.isDefinitiveImpact.head(5)
print("DefinitiveImpact的发生频率：",train_df.isDefinitiveImpact.value_counts()[0]/train_df.isDefinitiveImpact.value_counts()[1])
train_df.isDefinitiveImpact.value_counts()


# Train/Test player tracking.csv
# 不用考虑test_player_tracking.csv因为这是需要submission的

In [ ]:
train_tracking_df.gameKey.nunique()

In [ ]:
train_tracking_df.playID.nunique()

In [ ]:
# 查看祯数
frame_df = train_tracking_df.groupby(['gameKey', 'playID'])['time'].nunique().reset_index()
print("总祯数：",frame_df.time.sum())
print("最大祯数：",frame_df.time.max(),"最小祯数：",frame_df.time.min())
frame_df.head(3)

In [ ]:
# 画一下frame的图
sns.displot(frame_df.time)

# 分析snap的影响

In [ ]:
# Dividing 22 is considering the total player number 22
len(train_tracking_df.query('event == "ball_snap"'))//22

In [ ]:
train_tracking_df.query('event == "ball_snap"').playID.value_counts() / 22

In [ ]:
train_tracking_df.query('playID == 82 & event == "ball_snap"').head(1)

In [ ]:
# We can see that the player is not moving while 15 seconds flow
train_tracking_df.query('playID == 82 and snap_offset < 0 and player == "H97"')

In [ ]:
#几乎没有移动
before_snap_df = train_tracking_df.query('playID == 82 and snap_offset < 0 and player == "H97"')
before_snap_df.s.min(), before_snap_df.s.max()

In [ ]:
sns.displot(before_snap_df.s);

In [ ]:
# 开始移动
after_snap_df = train_tracking_df.query('playID == 82 and snap_offset > 0 and player == "H97"')
after_snap_df.s.min(), after_snap_df.s.max()

In [ ]:
sns.displot(after_snap_df.s);

# 标签信息是否总是每时每刻跟踪22名球员？
# 并不是，大部分是的，但是有时候会少于22人

In [ ]:
train_tracking_df.groupby(['playID', 'time']).count()['player'].value_counts()

In [ ]:
player_n_df = train_tracking_df.groupby(['playID', 'time']).count()['player'].reset_index()
player_n_df.head(5)

In [ ]:
player_n_df.query('player == 11')
#找到只有那个时间点

In [ ]:
train_tracking_df.query('time == "2018-10-29 02:22:44.099000+00:00"')
# 的确是11个球员

# image_labels.csv分析

In [ ]:
# bbox的个数
len(image_df)

In [ ]:
# 总图片数
image_df.image.nunique()

In [ ]:
# This feature doesn't seem useful to our competition
image_df.label.hist();

In [ ]:
image_df.groupby('image').count().head(5)

In [ ]:
# remove "frame", ".jpg" to compare with the whole frame
image_df.image = image_df.image.str.replace('frame', '')
image_df.image = image_df.image.str.replace('.jpg', '')
image_df.head(5)

In [ ]:
## 辅助图片
aux_images = image_df.image

How many games match?
41场比赛。
38场比赛有一场比赛
3场比赛有两场比赛

In [ ]:
aux_games_set = set(aux_images.str.split('_').str[0])
train_games_set = set(train_df.gameKey.astype(str))

In [ ]:
len(aux_games_set), len(train_games_set)

In [ ]:
common_games = list(aux_games_set & train_games_set)
print(common_games)

In [ ]:
# 38 games matches between one play only games and common games
one_play_only = set([str(game) for game in play_per_game[1]])
len(set(common_games).intersection(one_play_only))

In [ ]:
# 3 two play only games matches with common games
two_play_only = set([str(game) for game in play_per_game[2]])
len(set(common_games).intersection(two_play_only))

How many plays match? Only 1 matches. 58000_001306. Besides image_labels.csv has lot's of various plays!

In [ ]:
train_df.head(2)

In [ ]:
aux_play_set = set(aux_images.str.replace('_\d*$', ''))
train_play_set = set(train_df.video_frame.str.replace('_\d*$', ''))

In [ ]:
len(aux_play_set), len(train_play_set)

In [ ]:
aux_play_set & train_play_set

How many frames match? Only 2 matches. image_labels.csv has mostly one image per play

In [ ]:
aux_frame_set = set(aux_images)
train_frame_set = set(train_df.video_frame)

In [ ]:
len(aux_frame_set), len(train_frame_set)

In [ ]:
# both are not subset for each other
aux_frame_set.issubset(train_frame_set), train_frame_set.issubset(aux_frame_set)

In [ ]:
aux_frame_set & train_frame_set

# Train / Test baseline helmets.csv分析

In [ ]:
train_predict_df['video'] = train_predict_df.video_frame.str.replace('_\d*$', '')
test_predict_df['video'] = test_predict_df.video_frame.str.replace('_\d*$', '')

In [ ]:
train_predict_df.head(5)

In [ ]:
test_predict_df.head(5)

In [ ]:
# prediction is for the train set that we are using
set(train_predict_df.video.unique()) == set(train_df.video_frame.str.replace('_\d*$', ''))

In [ ]:
set(test_predict_df.video.unique())

In [ ]:
test_videos.sort()
test_videos

How many bbox was predicted for 1 frame?

In [ ]:
# 82 bbox was predicted in max for 1 frame
train_predict_df.groupby('video_frame').count()['left'].max()

In [ ]:
# 2 bbox was predicted in max for 1 frame
train_predict_df.groupby('video_frame').count()['left'].min()

In [ ]:
# bbox number doesn't seem quite right
sns.displot(train_predict_df.groupby('video_frame').count()['left']);

In [ ]:
train_predict_df.groupby('video_frame').count().query('left == 82')

In [ ]:
frame_bbox(train_predict_df, '58094_002819_Sideline_169')

In [ ]:
train_predict_df.groupby('video_frame').count().query('left == 2')

In [ ]:
frame_bbox(train_predict_df, '57584_000336_Sideline_454')

In [ ]:
# 查看置信度与标注的关系
filter_df = train_predict_df.copy().query('conf > 0.75')
filter_df

In [ ]:
filter_df = train_predict_df.copy().query('conf < 0.35')
filter_df